In [ ]:
#UP2009045
import flwr as fl
import New_Utils
import pandas as pd
import numpy as np
from flwr.common import NDArrays, Scalar
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt
from typing import Dict, Tuple, Optional
import joblib

def fit_round(server_round: int) -> Dict:
    """Send round number to client."""
    return {"server_round": server_round}

def get_evaluate_fn(model: LogisticRegression):
    """Return an evaluation function for server-side evaluation."""
    (X_train, y_train),(X_test, y_test) = New_Utils.load_Data()

    def evaluate(
        server_round: int, parameters: NDArrays, config: Dict[str, Scalar]
    ) -> Optional[Tuple[float, Dict[str, Scalar]]]:
        New_Utils.set_model_params(model, parameters)
        predictions = model.predict(X_test)
        probabilities = model.predict_proba(X_test)[:, 1]
        loss = log_loss(y_test, probabilities)
        accuracy = model.score(X_test, y_test)
        shape = model.coef_.shape, model.intercept_.shape

        # Calculate and plot confusion matrix only in the last round
        if server_round == 10:
            cm = confusion_matrix(y_test, predictions, normalize='true')
            disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
            disp.plot()
            plt.show()

            # Calculate Precision, Recall, F1 Score, and ROC AUC
            precision = precision_score(y_test, predictions)
            recall = recall_score(y_test, predictions)
            f1 = f1_score(y_test, predictions)
            roc_auc = roc_auc_score(y_test, probabilities)

            # Plot Precision, Recall, F1 Score, and ROC AUC
            metrics = [loss, accuracy, precision, recall, f1, roc_auc]
            metric_names = ['Loss', 'Accuracy', 'Precision', 'Recall', 'F1 Score', 'ROC AUC']
            plt.bar(metric_names, metrics)
            plt.title('Model Metrics')
            plt.show()

            # Plot ROC curve
            fpr, tpr, _ = roc_curve(y_test, probabilities)
            plt.plot(fpr, tpr)
            plt.title('ROC Curve')
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            plt.show()

            # Plot Precision-Recall curve
            precision, recall, _ = precision_recall_curve(y_test, probabilities)
            plt.plot(recall, precision)
            plt.title('Precision-Recall Curve')
            plt.xlabel('Recall')
            plt.ylabel('Precision')
            plt.show()
        return loss, {"accuracy": accuracy}
        
    return evaluate

# Start Flower server for three rounds of federated learning
if __name__ == "__main__":
    model = joblib.load('5XV.joblib') #remove this line if you want to train the model again replace it with model = LogisticRegression()
    New_Utils.set_initial_params(model)
    strategy = fl.server.strategy.FedAvg(
        min_available_clients=30,
        evaluate_fn=get_evaluate_fn(model),
        on_fit_config_fn=fit_round
    )
    fl.server.start_server(server_address="127.0.0.1:8080", strategy=strategy, config=fl.server.ServerConfig(num_rounds=10))
joblib.dump(model,'5XV.joblib')